# Basic import

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rosbag
import cv2
import numpy as np
import pandas as pd
from cv_bridge import CvBridge, CvBridgeError

import re
from detectionExtraction import *
from trackExtraction import *

# Bag management

In [ ]:
# Create bag to process
bag = rosbag.Bag('../../bags/sabrine.bag')
print bag

In [ ]:
# create bridge between CV and ROS
br = CvBridge()

# Dataframes creation
#### Creation of the RGB dataframe

In [ ]:
# Create array for rgb frames
rgbTime = []
rgbSeq = []
for topic, msg, t in bag.read_messages(topics=['/kinect2_head/rgb/image']):
    time = msg.header.stamp
    rgbTime.append(int(str(time)))
    seq = msg.header.seq
    rgbSeq.append(int(seq))
    msg.encoding = "bgr8"
    cv_image = br.imgmsg_to_cv2(msg, desired_encoding="bgr8")
    fileNumber = str("%03d"%seq)
    cv2.imwrite('/home/sabrine/notebook/framesRGB/' + fileNumber + '.jpg',cv_image)

In [ ]:
data = np.asarray([np.asarray(rgbTime), np.asarray(rgbSeq)]).T

In [ ]:
rgb = pd.DataFrame(data=data)

In [ ]:
rgb.columns = ['rgbTime', 'rgbSeq']

In [ ]:
rgb

#### Creation of the detection dataframe

In [ ]:
detect = detection_extraction('../../bags/sabrine.bag')

In [ ]:
detect.detectionTimestamp.is_unique and detect.detecSeq.is_unique

#### Creation of the tracking dataframe

In [ ]:
tra = track_extraction('../../bags/sabrine.bag')

In [ ]:
tra.traSeq.is_unique

In [ ]:
tra[['trackTimestamp', 'traSeq', 'trackId', 'trackX', 'trackY', 'trackW', 'trackH']] = tra[['trackTimestamp', 'traSeq', 'trackId', 'trackX', 'trackY', 'trackW', 'trackH']].astype(int)

# Synchronization

In [ ]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

#### Synchronize detections and frames

In [ ]:
# find closest rgb frames for each detections
match_frame= []
for index, row in detect.iterrows():
    #print index
    rgbSeqMatched = find_nearest(np.asarray(rgbTime), int(str(row.detectionTimestamp)))
    match_frame.append(rgbSeqMatched)

In [ ]:
detect['closestFrameTime'] = match_frame

In [ ]:
# merge both information in the same dataframe
synchronize = pd.merge(detect, rgb, how='inner', left_on='closestFrameTime', right_on='rgbTime',
         left_index=False, right_index=False, sort=True,
         suffixes=('_detect', '_rgb'), copy=True, indicator=False)

In [ ]:
# drop unusefull columns
synchronize.drop(['rgbTime', 'closestFrameTime'], axis=1, inplace=True)

In [ ]:
synchronize

In [ ]:
synchronize.detectionTimestamp.is_unique and synchronize.detecSeq.is_unique

#### Synchronize traking and detections

In [ ]:
tra

In [ ]:
# find closest detections for each tracks
match_d= []

for t in tra.trackTimestamp.unique():
    dSeqMatched = find_nearest(detect.detectionTimestamp.astype(int), int(t))
    match_d.append(dSeqMatched)
#for index, row in tra.iterrows():
    #print index
    #dSeqMatched = find_nearest(detect.detectionTimestamp.astype(int), 
    #                             int(row.trackTimestamp))
    #match_d.append(dSeqMatched)

In [ ]:
len(match_d)

In [ ]:
type(match_d)

In [ ]:
data = np.asarray([np.asarray(match_d, dtype=int), np.asarray(tra.trackTimestamp.unique(), dtype=int)]).T

In [ ]:
temp = pd.DataFrame(data=data)

In [ ]:
temp.columns = ['match_d', 'tTimeUnique']

In [ ]:
temp

In [ ]:
# merge both information in the same dataframe
synchronize2 = pd.merge(temp, tra, how='right', left_on='tTimeUnique', right_on='trackTimestamp',
         left_index=False, right_index=False, sort=True,
         suffixes=('_t', '_d'), copy=True, indicator=False)

As I merge using the right df as fix, it create sometime twice information for the same id in the same frame

In [ ]:
synchronize2

In [ ]:
synchronize2.tTimeUnique == synchronize2.trackTimestamp

In [ ]:
# drop unusefull columns
synchronize2.drop(['tTimeUnique'], axis=1, inplace=True)

#### Synchronize tracks and frames

In [ ]:
# merge both information in the same dataframe
synchronize3 = pd.merge(synchronize2, synchronize, how='left', left_on='match_d', right_on='detectionTimestamp',
         left_index=False, right_index=False, sort=True,
         suffixes=('_t', '_d'), copy=True, indicator=False)

In [ ]:
synchronize3.trackTimestamp.astype(int) == synchronize3.detectionTimestamp.astype(int)

In [ ]:
tuples = list(zip(np.asarray(synchronize3.traSeq), np.asarray(synchronize3.trackId)))

In [ ]:
index = pd.MultiIndex.from_tuples(tuples, names=['seq', 'id'])

In [ ]:
synchronize3.index = index

In [ ]:
synchronize3.index.is_unique

In [ ]:
# drop unusefull columns
synchronize3.drop(['match_d', 'detectionTimestamp', 'detecSeq', 'trackTimestamp', 'match_d'], axis=1, inplace=True)

In [ ]:
synchronize3

The first rgbSeq is the 42th.

It is normal as before the 42th frame we had no traking information

In [ ]:
#Need to adapt the coordinate to the high quality picture
synchronize3.trackX = synchronize3.trackX.astype(int) * 1.7 * 2
synchronize3.trackY = synchronize3.trackY.astype(int) * 1.0 * 2
synchronize3.trackH = synchronize3.trackH.astype(int) * 1.0 * 2
synchronize3.trackW = synchronize3.trackW.astype(int) * 1.7 * 2

In [ ]:
synchronize3.to_pickle('synchronizeS')